# 0_Challenges analysis

**Objective:** Assign accurately demographics data to individual households in Germany


Here, we will start to "brainstorm" all the possible challenges for the mini-project. Let's start with the data we can think of:
1. List of postal codes (PLZ) in Germany
1. List of municipal codes (regional key) more details can be found [HERE](!https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Glossar/regionalschluessel.html)
1. Statistics from Regionaldatenbank (demographics data) with the most granular level of data is municipal. Population distribution by ages and genders [HERE](!https://www.regionalstatistik.de/genesis//online/data?operation=table&code=12411-02-03-5-B&levelindex=1&levelid=1604855547818)
1. Building objects from OpenStreetMap. Can be query by postal code-level or municipal-level.

## Challenges

### Classify building objects

### Assign demographics data

There are 3 possible cases when matching postal codes data with regional-key data:
1. 1 PLZ = 1 regional-key. This is the easiest case since we can just divide the demographics data (municipal-level) equally to all residential buildings in the postal code.
1. 1 Regional-key contains multiple PLZs. For example: Aalen, Stadt in Baden-Wüttemberg which has regional-key as 081365001088 but contains 5 PLZ from 73430 to 73434
1. 1 PLZ is laid within multiple regional-keys. For example: PLZ 37339 is consisted of 7 municipals (last 3 digits change) 160615001003/015/026/031/094/103/114

## Initialization

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import importlib
import logging 

from datetime import datetime
timestamp = datetime.now().strftime("%d%m%y_%H%M")

### Load custom modules

In [2]:
pkg_path = '../src/'

sys.path.append(pkg_path)
import data_acquisition as da

In [3]:
# Reload module (incase new update)
importlib.reload(da)

<module 'data_acquisition' from '../src\\data_acquisition.py'>

## Inputs / Outputs

In [4]:
# Paths
pop_gmd_path = '../data/01_raw/pop_gem_de.csv'
plz_gmd_path = '../data/01_raw/zuordnung_plz_ort_landkreis.csv'

In [5]:
# Postal code list with population data (2011)
pop_plz_csv = '../data/01_raw/plz_einwohner.csv'
pop_plz_de = pd.read_csv(pop_plz_csv, dtype={'plz': str, 'einwhoner': int})  

In [6]:
pop_plz_de.shape

(8168, 2)

In [7]:
pop_plz_de.head()

,plz,einwohner
0,01067,11957
1,01069,25491
2,01097,14811
3,01099,28021
4,01108,5876


PLZ (postal code) - Gemeinden (municipal) - Landkreis (district) map

In [8]:
plz_gmd_lk = pd.read_csv(plz_gmd_path)
plz_gmd_lk.rename(columns = {'ags':'Regionalschluessel'}, inplace = True)

In [9]:
plz_gmd_lk.shape

(12883, 6)

In [10]:
print(f'Total number of unique PLZ: {len(plz_gmd_lk.plz.drop_duplicates())}')
print(f'Total number of unique Gemeinden (municipal): {len(plz_gmd_lk.Regionalschluessel.drop_duplicates())}')

Total number of unique PLZ: 8168
Total number of unique Gemeinden (municipal): 10797


In [11]:
# 1 RS = multiple PLZ
plz_gmd_lk[plz_gmd_lk.duplicated(['Regionalschluessel'])]

,osm_id,Regionalschluessel,ort,plz,landkreis,bundesland
3,62564,5334002,Aachen,52064,Städteregion Aachen,Nordrhein-Westfalen
4,62564,5334002,Aachen,52066,Städteregion Aachen,Nordrhein-Westfalen
5,62564,5334002,Aachen,52068,Städteregion Aachen,Nordrhein-Westfalen
6,62564,5334002,Aachen,52070,Städteregion Aachen,Nordrhein-Westfalen
7,62564,5334002,Aachen,52072,Städteregion Aachen,Nordrhein-Westfalen
...,...,...,...,...,...,...
12873,397076,14524330,Zwickau,8058,Landkreis Zwickau,Sachsen
12874,397076,14524330,Zwickau,8060,Landkreis Zwickau,Sachsen
12875,397076,14524330,Zwickau,8062,Landkreis Zwickau,Sachsen
12876,397076,14524330,Zwickau,8064,Landkreis Zwickau,Sachsen


In [12]:
# 1 PLZ = multiple RS
plz_gmd_lk[plz_gmd_lk.duplicated(['plz'])]

,osm_id,Regionalschluessel,ort,plz,landkreis,bundesland
35,1246405,7134002,Achtelsbach,55767,Landkreis Birkenfeld,Rheinland-Pfalz
63,448765,1061003,Agethorst,25560,Kreis Steinburg,Schleswig-Holstein
94,1406977,1054004,Ahrenviölfeld,25885,Kreis Nordfriesland,Schleswig-Holstein
133,958104,9275114,Aldersbach,94501,Landkreis Passau,Bayern
220,418895,7132501,Altenkirchen (Westerwald),57610,Landkreis Altenkirchen,Rheinland-Pfalz
...,...,...,...,...,...,...
12863,1257089,7235153,Züsch,54422,Landkreis Trier-Saarburg,Rheinland-Pfalz
12865,1515590,13074091,Züsow,23992,Landkreis Nordwestmecklenburg,Mecklenburg-Vorpommern
12866,1433754,13075154,Züssow,17495,Landkreis Vorpommern-Greifswald,Mecklenburg-Vorpommern
12869,1171875,7232138,Zweifelscheid,54673,Eifelkreis Bitburg-Prüm,Rheinland-Pfalz


Population distribution per municipal

In [13]:
# Population distribution municipal-level
pop_gmd_df = pd.read_csv(pop_gmd_path, sep=';',
                         encoding='iso-8859-1',
                         header=[0, 1]
                        )

# Merge first 2 rows as column names
pop_gmd_df.columns = pop_gmd_df.columns.map('_'.join)

# Rename first 2 columns
pop_gmd_df.rename(columns={pop_gmd_df.columns[0]:'Regionalschluessel',
                           pop_gmd_df.columns[1]:'Gemeinden'},
                  inplace=True)

# Take out only total population per gender
pop_gmd_df = pop_gmd_df[['Regionalschluessel',
                         'Gemeinden',
                         'Insgesamt_Insgesamt',
                         'männlich_Insgesamt',
                         'weiblich_Insgesamt']]

In [14]:
pop_gmd_df.shape

(13556, 5)

In [15]:
pop_gmd_df.head()

,Regionalschluessel,Gemeinden,Insgesamt_Insgesamt,männlich_Insgesamt,weiblich_Insgesamt
0,1001000,"Flensburg, krsfr. Stadt",89504,44599,44905
1,1002000,"Kiel, krsfr. Stadt",247548,120566,126982
2,1003000,"Lübeck, krsfr. Stadt",217198,104371,112827
3,1004000,"Neumünster, krsfr. Stadt",79487,39241,40246
4,1051001,Albersdorf,3670,1800,1870


In [16]:
pop_gmd_df[pop_gmd_df.Regionalschluessel==7232003]

,Regionalschluessel,Gemeinden,Insgesamt_Insgesamt,männlich_Insgesamt,weiblich_Insgesamt
4305,7232003,Altscheid,90,46,44


In [17]:
print(f'Total number of unique Gemeinden (municipal): {len(pop_gmd_df.Regionalschluessel.drop_duplicates())}')

Total number of unique Gemeinden (municipal): 13556


## EDA

In [51]:
# Different between 2 data sets about Gemeinden
a = set(pop_gmd_df.Regionalschluessel.drop_duplicates())

In [52]:
b = set(plz_gmd_lk.Regionalschluessel.drop_duplicates())

In [53]:
# anti-join to get differences between 2 datasets
c = (a ^ b)
len(c)

2761

In [54]:
plz_gmd_lk[plz_gmd_lk.Regionalschluessel.isin(c)]

,osm_id,Regionalschluessel,ort,plz,landkreis,bundesland
11974,1808966,3358024,Walsrode,29664,Landkreis Heidekreis,Niedersachsen
11975,1808966,3358024,Walsrode,29699,Landkreis Heidekreis,Niedersachsen


In [55]:
pop_gmd_df.Insgesamt_Insgesamt = pop_gmd_df.Insgesamt_Insgesamt.astype(int, errors='ignore')

In [57]:
pop_gmd_df[(pop_gmd_df.Regionalschluessel.isin(c)) & 
           (pop_gmd_df.Insgesamt_Insgesamt.isin(['-0','.']) == False)].reset_index(drop = True)

,Regionalschluessel,Gemeinden,Insgesamt_Insgesamt,männlich_Insgesamt,weiblich_Insgesamt
0,3358004,Bomlitz,6970,3458,3512
1,3358022,Walsrode,23068,11491,11577
2,6633021,Oberweser,3158,1591,1567
3,6633027,Wahlsburg,2044,999,1045
4,7339027,Heidesheim am Rhein,7619,3795,3824
...,...,...,...,...,...
242,16077002,Altkirchen,970,503,467
243,16077004,Drogen,127,65,62
244,16077029,Lumpzig,494,231,263
245,16077037,Nöbdenitz,860,424,436
